# Convolutional Neural Network

### Importing the libraries

In [26]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore


Data Preprocessing

### Preprocessing the Training set

In [27]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 8000 images belonging to 2 classes.


The ImageDataGenerator applies transformations like rescaling, shearing, zooming, and horizontal flipping to augment the images.

flow_from_directory loads the images from a specified directory, resizes them to 64x64 pixels, processes them in batches of 32, and returns them with binary class labels (0 or 1) for training.

### Preprocessing the Test set

In [28]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary') #following same approach as that of training set

Found 2000 images belonging to 2 classes.


Building the CNN

### Initialising the CNN

In [29]:
cnn = tf.keras.models.Sequential()#will process data and make predictions based on learned features

### Step 1 - Convolution

In [30]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))
#This convolutional layer will help the model start learning low-level features from the input images, such as edges and textures.

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


### Step 2 - Pooling

In [31]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))
#pooling operation helps in reducing the dimensions of the feature map, making the model more computationally efficient and less prone to overfitting

### Adding a second convolutional layer

Adding them to detect low level features in early layers 
learns more detailed patterns in deep layers.

In [32]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [33]:
cnn.add(tf.keras.layers.Flatten())#converts multiple dimentions datas into 1D vector and it is needed before passing dense layers

### Step 4 - Full Connection

In [34]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
#After the CNN extracts features (edges, shapes, textures, etc.), 
# we need to combine those features and make a decision
#This dense layer learns how to interpret the extracted features and make sense of them.

### Step 5 - Output Layer

In [35]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
#this line wraps up the model. 
# All the earlier layers extract and learn features, and this final layer makes the decision

### Part 3 - Training the CNN

### Compiling the CNN

In [36]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
#compiles CNN model, this line is telling it how to learn and what to learn and how to evaluate its performance.

### Training the CNN on the Training set and evaluating it on the Test set

In [ ]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 25)
#takes  images, runs them through the network, compares predictions with actual labels, and adjusts the weights to reduce error

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 55ms/step - accuracy: 0.5615 - loss: 0.6878 - val_accuracy: 0.6130 - val_loss: 0.6585
Epoch 2/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 51ms/step - accuracy: 0.6764 - loss: 0.6028 - val_accuracy: 0.6890 - val_loss: 0.5853
Epoch 3/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 14s 58ms/step - accuracy: 0.7158 - loss: 0.5584 - val_accuracy: 0.7195 - val_loss: 0.5503
Epoch 4/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.7418 - loss: 0.5154 - val_accuracy: 0.7620 - val_loss: 0.4864
Epoch 5/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 50ms/step - accuracy: 0.7549 - loss: 0.4917 - val_accuracy: 0.7620 - val_loss: 0.5005
Epoch 6/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 13s 52ms/step - accuracy: 0.7787 - loss: 0.4736 - val_accuracy: 0.7815 - val_loss: 0.4741
Epoch 7/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 49ms/step - accuracy: 0.7811 - loss: 0.4652 - val_accuracy: 0.7460 - val_loss: 0.5520
Epoch 8/25
250/250 ━━━━━━━━━━━━━━━━━━━━ 12s 47ms/step - accuracy: 0.7854 - loss: 0.4452 - 

## Part 4 - Making a single prediction

In [ ]:
import numpy as np
from keras.preprocessing import image # type: ignore
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image / 255.0  # Normalization is important I tried without normalization and it was giving same output for both images
result = cnn.predict(test_image)
training_set.class_indices
if result[0][0] > 0.5:
    prediction = 'dog'
else:
    prediction = 'cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step


In [ ]:
print(prediction)

cat
